In [1]:
# import pandas as pd
# import numpy as np
# from typing import List

# def hierarchical_max_sort(df: pd.DataFrame, 
#                          value_col: str,
#                          index_levels: List[str] = None) -> pd.DataFrame:
#     """
#     Sort a DataFrame hierarchically by maximum values at each level of the index.
    
#     Args:
#         df: DataFrame with multi-index
#         value_col: Name of the column to sort by
#         index_levels: List of index level names. If None, uses all levels in order
    
#     Returns:
#         Sorted DataFrame
#     """
#     if index_levels is None:
#         index_levels = list(df.index.names)
    
#     df_temp = df.copy()
    
#     # Add maximum value columns for each level
#     for i in range(len(index_levels)):
#         level_name = f'level{i+1}_max'
#         group_levels = index_levels[:i+1]
#         df_temp[level_name] = df.groupby(group_levels)[value_col].transform('max')
    
#     # Sort by all max columns
#     max_cols = [f'level{i+1}_max' for i in range(len(index_levels))]
#     df_sorted = df_temp.sort_values(
#         max_cols,
#         ascending=[False] * len(max_cols)
#     )[value_col].to_frame()
    
#     return df_sorted

# # Example usage:
# if __name__ == "__main__":
#     # Sample data
#     data = {
#         'i1': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3],
#         'i2': ['a', 'a', 'b', 'b', 'a', 'a', 'b', 'a', 'a', 'b', 'b', 'b'],
#         'i3': ['x', 'y', 'z', 'x', 'x', 'y', 'z', 'x', 'y', 'x', 'y', 'z'],
#         'v': [10, 5, 20, 15, 3, 7, np.nan, 30, 25, 8, 12, 6]
#     }
    
#     # Create DataFrame and set multi-index
#     df = pd.DataFrame(data)
#     df = df.set_index(['i1', 'i2', 'i3'])
    
#     # Sort the DataFrame
#     df_sorted = hierarchical_max_sort(df, 'v')
    
#     print("Original DataFrame:")
#     print(df)
#     print("\nSorted DataFrame:")
#     print(df_sorted)

In [2]:
import pandas as pd
import numpy as np
from typing import List

def hierarchical_max_sort(df: pd.DataFrame, 
                         value_col: str,
                         index_levels: List[str] = None) -> pd.DataFrame:
    """
    Sort a DataFrame hierarchically by maximum values at each level of the index,
    preserving all columns.
    
    Args:
        df: DataFrame with multi-index
        value_col: Name of the column to sort by
        index_levels: List of index level names. If None, uses all levels in order
    
    Returns:
        Sorted DataFrame with all original columns
    """
    if index_levels is None:
        index_levels = list(df.index.names)
    
    df_temp = df.copy()
    
    # Add maximum value columns for each level
    for i in range(len(index_levels)):
        level_name = f'level{i+1}_max'
        group_levels = index_levels[:i+1]
        df_temp[level_name] = df.groupby(group_levels)[value_col].transform('max')
    
    # Sort by all max columns
    max_cols = [f'level{i+1}_max' for i in range(len(index_levels))]
    df_sorted = df_temp.sort_values(
        max_cols,
        ascending=[False] * len(max_cols)
    )
    
    # Drop temporary sorting columns and return original columns
    df_sorted = df_sorted.drop(columns=max_cols)
    
    return df_sorted

# Test the generalized function
if __name__ == "__main__":
    # Sample data with additional columns
    data = {
        'i1': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3],
        'i2': ['a', 'a', 'b', 'b', 'a', 'a', 'b', 'a', 'a', 'b', 'b', 'b'],
        'i3': ['x', 'y', 'z', 'x', 'x', 'y', 'z', 'x', 'y', 'x', 'y', 'z'],
        'i4': [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
        'i5': ['p', 'q', 'p', 'q', 'p', 'q', 'p', 'q', 'p', 'q', 'p', 'q'],
        'v': [10, 5, 20, 15, 3, 7, np.nan, 30, 25, 8, 12, 6],
        'other_col1': range(12),
        'other_col2': ['A'] * 6 + ['B'] * 6
    }
    
    # Create DataFrame and set multi-index
    df = pd.DataFrame(data)
    df = df.set_index(['i1', 'i2', 'i3', 'i4', 'i5'])
    
    # Sort using only i1, i2, i3 but preserve all columns
    df_sorted = hierarchical_max_sort(df, 'v', index_levels=['i1', 'i2', 'i3'])
    
    print("Original DataFrame:")
    print(df)
    print("\nSorted DataFrame (using i1, i2, i3, preserving all columns):")
    print(df_sorted)

Original DataFrame:
                   v  other_col1 other_col2
i1 i2 i3 i4 i5                             
1  a  x  1  p   10.0           0          A
      y  2  q    5.0           1          A
   b  z  1  p   20.0           2          A
      x  2  q   15.0           3          A
2  a  x  1  p    3.0           4          A
      y  2  q    7.0           5          A
   b  z  1  p    NaN           6          B
3  a  x  2  q   30.0           7          B
      y  1  p   25.0           8          B
   b  x  2  q    8.0           9          B
      y  1  p   12.0          10          B
      z  2  q    6.0          11          B

Sorted DataFrame (using i1, i2, i3, preserving all columns):
                   v  other_col1 other_col2
i1 i2 i3 i4 i5                             
3  a  x  2  q   30.0           7          B
      y  1  p   25.0           8          B
   b  y  1  p   12.0          10          B
      x  2  q    8.0           9          B
      z  2  q    6.0          11      

In [3]:
import pandas as pd
import numpy as np

# Test with more complex data
data = {
    'i1': [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3],
    'i2': ['a', 'a', 'b', 'b', 'a', 'a', 'b', 'a', 'a', 'b', 'b', 'b'],
    'i3': ['x', 'y', 'z', 'x', 'x', 'y', 'z', 'x', 'y', 'x', 'y', 'z'],
    'i4': [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
    'i5': ['p', 'q', 'p', 'q', 'p', 'q', 'p', 'q', 'p', 'q', 'p', 'q'],
    'v': [10, 5, 20, 15, 3, 7, np.nan, 30, 25, 8, 12, 6]
}

# Create DataFrame with all indices
df_full = pd.DataFrame(data)
df_full = df_full.set_index(['i1', 'i2', 'i3', 'i4', 'i5'])

# Test existing function with specific levels
df_sorted = hierarchical_max_sort(df_full, 'v', index_levels=['i1', 'i2', 'i3'])

print("Original DataFrame:")
print(df_full)
print("\nSorted DataFrame (using only i1, i2, i3):")
print(df_sorted)

# Also test without specifying levels (should use all 5 levels)
df_sorted_all = hierarchical_max_sort(df_full, 'v')

print("\nSorted DataFrame (using all levels):")
print(df_sorted_all)

Original DataFrame:
                   v
i1 i2 i3 i4 i5      
1  a  x  1  p   10.0
      y  2  q    5.0
   b  z  1  p   20.0
      x  2  q   15.0
2  a  x  1  p    3.0
      y  2  q    7.0
   b  z  1  p    NaN
3  a  x  2  q   30.0
      y  1  p   25.0
   b  x  2  q    8.0
      y  1  p   12.0
      z  2  q    6.0

Sorted DataFrame (using only i1, i2, i3):
                   v
i1 i2 i3 i4 i5      
3  a  x  2  q   30.0
      y  1  p   25.0
   b  y  1  p   12.0
      x  2  q    8.0
      z  2  q    6.0
1  b  z  1  p   20.0
      x  2  q   15.0
   a  x  1  p   10.0
      y  2  q    5.0
2  a  y  2  q    7.0
      x  1  p    3.0
   b  z  1  p    NaN

Sorted DataFrame (using all levels):
                   v
i1 i2 i3 i4 i5      
3  a  x  2  q   30.0
      y  1  p   25.0
   b  y  1  p   12.0
      x  2  q    8.0
      z  2  q    6.0
1  b  z  1  p   20.0
      x  2  q   15.0
   a  x  1  p   10.0
      y  2  q    5.0
2  a  y  2  q    7.0
      x  1  p    3.0
   b  z  1  p    NaN
